In [9]:
import pandas as pd
from json import loads, dumps


In [13]:
datas = ['mpqa']

df1 = [pd.read_json(f"../../final_data/{i}/train_convert.json") for i in datas]
df = pd.concat(df1)
df.reset_index(drop=True)

,sent_id,raw_words,words,aspects,holder,opinions
0,20020415/21.31.55-5725-0,NA Two IDF soldiers were lightly wounded in gu...,"[NA, Two, IDF, soldiers, were, lightly, wounde...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
1,20020415/21.31.55-5725-1,NA They were evacuated to Jerusalem 's Hadassa...,"[NA, They, were, evacuated, to, Jerusalem, 's,...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
2,20020415/21.31.55-5725-2,NA Sources in Bethlehem reported that the IDF ...,"[NA, Sources, in, Bethlehem, reported, that, t...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
3,20020415/21.31.55-5725-3,NA The Palestinians also reported that the IDF...,"[NA, The, Palestinians, also, reported, that, ...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
4,20020509/22.11.01-7259-0,"NA Shanghai , May 9 ( XINHUA )--","[NA, Shanghai, ,, May, 9, (, XINHUA, )--]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
...,...,...,...,...,...,...
1814,20011112/20.33.31-29984-10,"NA And it is possible that , at some given tim...","[NA, And, it, is, possible, that, ,, at, some,...","[{'index': 0, 'from': 15, 'to': 16, 'term': 'i...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 14, 'to': 18, 'term': 'h..."
1815,20011112/20.33.31-29984-13,"NA "" The US authorities would not like to have...","[NA, "", The, US, authorities, would, not, like...","[{'index': 0, 'from': 9, 'to': 28, 'term': 'ha...","[{'index': 0, 'from': 30, 'to': 31, 'term': 'h...","[{'index': 0, 'from': 5, 'to': 8, 'term': 'wou..."
1816,20011112/20.33.31-29984-21,"NA He emphasized : "" Now Mexico is positioned ...","[NA, He, emphasized, :, "", Now, Mexico, is, po...","[{'index': 0, 'from': 31, 'to': 34, 'term': 't...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'He'}]","[{'index': 0, 'from': 26, 'to': 31, 'term': 't..."
1817,20011112/20.33.31-29984-22,"NA In terms of market , the Mexican exports do...","[NA, In, terms, of, market, ,, the, Mexican, e...","[{'index': 0, 'from': 20, 'to': 21, 'term': 'O...","[{'index': 0, 'from': 25, 'to': 32, 'term': 'M...","[{'index': 0, 'from': 17, 'to': 19, 'term': 't..."


In [22]:
df1 = pd.read_json('ddx.json')
df2 = pd.read_json('ddx1.json')

In [24]:
df1[df1["opinions"] == '[]']

,sent_id,raw_words,words,aspects,holder,opinions


In [21]:
df.to_json(r'ddx1.json', orient='records')

In [1]:
from tqdm import tqdm
import os, json

NULL_TOKEN = 'NA'
START_WORD = 'NA '
NULL_INDEX = 0

def char2word_offset(sent, start_end_str):
    start, end = [int(i) for i in start_end_str.split(':')]
    start, end = start+len(START_WORD), end+len(START_WORD)
    word_start, word_end = None, None
    space_count = 0
    for i in range(len(sent)):
        if sent[i]==' ': space_count += 1
        if i>=start and word_start is None: word_start = space_count
        if i>=end and word_start is not None: word_end = space_count; break
    if word_end is None and word_start is not None: word_end = space_count+1
    assert (type(word_start) == type(word_end))
    if word_start is None: word_start, word_end = NULL_INDEX, NULL_INDEX+1
    return int(word_start), int(word_end)

def format_convert(original_data, file, threshold=0):
    final_data = []
    blank_cnt = 0
    threshold *= len(original_data)
    if 't1' in file: print(f'{threshold} limit for null opinions')

    for row in tqdm(original_data):
        data = {}
        if 't1' in file:
            if blank_cnt <= threshold and len(row['opinions'])==0: blank_cnt += 1
            elif len(row['opinions'])==0: continue

        row['text'] = START_WORD + row['text']
        data['sent_id'] = row['sent_id']
        data['raw_words'] = row['text']
        data['words'] = data['raw_words'].split()
        aspects, holders, opinions = [], [], []

        for cnt, original_opinion in enumerate(row['opinions']):
            aspect, holder, opinion = {}, {}, {}
            aspect['index'], holder['index'], opinion['index'] = cnt, cnt, cnt

            ############################################# Aspect #############################################
            try:aspect['from'], aspect['to'] = char2word_offset(data['raw_words'], original_opinion['Target'][1][0])
            except: aspect['from'], aspect['to'] = NULL_INDEX, NULL_INDEX+1
            if aspect['from'] == aspect['to']-1 == NULL_INDEX:
                aspect['term'] = NULL_TOKEN
            else:
                aspect['term'] = original_opinion['Target'][0][0]

            try: aspect['Polarity'] = str(original_opinion['Polarity'])
            except: aspect['Polarity'] = NULL_TOKEN; print(original_opinion)
            try: aspect['Intensity'] = str(original_opinion['Intensity'])
            except: aspect['Intensity'] = NULL_TOKEN; print(original_opinion)

            ############################################# Holder #############################################
            try:holder['from'], holder['to'] = char2word_offset(data['raw_words'], original_opinion['Source'][1][0])
            except: holder['from'], holder['to'] = NULL_INDEX, NULL_INDEX+1
            if holder['from'] == holder['to']-1 == NULL_INDEX:
                holder['term'] = NULL_TOKEN
            else:
                holder['term']  = original_opinion['Source'][0][0]

            ############################################# Opinion #############################################
            try:opinion['from'], opinion['to'] = char2word_offset(data['raw_words'], original_opinion['Polar_expression'][1][0])
            except: opinion['from'], opinion['to'] = NULL_INDEX, NULL_INDEX+1
            if opinion['from'] == opinion['to']-1 == NULL_INDEX:
                opinion['term'] = NULL_TOKEN
            else:
                opinion['term'] = original_opinion['Polar_expression'][0][0]

            aspects.append(aspect)
            holders.append(holder)
            opinions.append(opinion)

        if len(aspects) == 0:
            tmp = {}; tmp['index'] = 0; tmp['from']=NULL_INDEX; tmp['to']=NULL_INDEX+1
            for k in 'term Polarity Intensity'.split(): tmp[k]=NULL_TOKEN
            aspects = [tmp]
        if len(holders) == 0:
            tmp = {}; tmp['index'] = 0; tmp['from']=NULL_INDEX; tmp['to']=NULL_INDEX+1
            for k in 'term'.split(): tmp[k]=NULL_TOKEN
            holders = [tmp]
        if len(opinions) == 0:
            tmp = {}; tmp['index'] = 0; tmp['from']=NULL_INDEX; tmp['to']=NULL_INDEX+1
            for k in 'term'.split(): tmp[k]=NULL_TOKEN
            opinions = [tmp]

        data['aspects'] = aspects
        data['holder'] = holders
        data['opinions'] = opinions
        final_data.append(data)

    return final_data

In [7]:
null_dict = {
    'mpqa': 0.1,
    'darmstadt_unis': 0.2,
    'norec': 0.3,
    'opener_en': 0,
    'multibooked_eu': 0.07,
    'multibooked_eu_crosslingual': 0.07,
    'multibooked_ca': 0.15,
    'opener_es': 0.06,
    "eng_combined": 0.1
}

for folder in os.listdir('../../final_data'):
    for file in os.listdir(f'../../final_data/{folder}'):
        if '1.' not in file: continue
        with open(f'../../final_data/{folder}/{file}', encoding="utf8") as f: data = json.load(f)

        data = format_convert(data, file, null_dict[folder])
        if 't1' in file: dest = 'train_convert.json'
        elif 'd1' in file: dest = 'dev_convert.json'
        elif 'f1' in file: dest = 'test_convert.json'
        print(f'../../final_data/{folder}/{dest}')
        with open(f'../../final_data/{folder}/{dest}', 'w') as f:
            json.dump(data, f)
        print()

../../final_data/darmstadt_unis/d1.json


100%|██████████| 232/232 [00:00<00:00, 108071.80it/s]


../../final_data/darmstadt_unis/dev_convert.json

../../final_data/darmstadt_unis/f1.json


100%|██████████| 318/318 [00:00<00:00, 79510.50it/s]


../../final_data/darmstadt_unis/test_convert.json

../../final_data/darmstadt_unis/t1.json
450.6 limit for null opinions


...

../../final_data/opener_es/dev_convert.json

../../final_data/opener_es/f1.json


100%|██████████| 410/410 [00:00<00:00, 24715.28it/s]


../../final_data/opener_es/test_convert.json

../../final_data/opener_es/t1.json
86.28 limit for null opinions


100%|██████████| 1438/1438 [00:00<00:00, 28753.58it/s]


../../final_data/opener_es/train_convert.json



